## Here we look at what the effect is of integrating all the marks

We want to see if we can pick out specific genes based on the latent space

## Read in from here


We want to get the weightings for different classes from this point onwards!


In [3]:
import pandas as pd

df_all = pd.read_csv('../data/df_all.csv')

In [29]:
df_all

,external_gene_name,VAE0,VAE1,VAE2,Log2FC FB,Log2FC MB,Log2FC HB,Log2FC SC,GroupLabel,H3K27me3,...,stomach_16.5-days_embryonic_H3K4me1_ChIP-seq_ENCFF814BNR_width,stomach_16.5-days_embryonic_H3K4me1_ChIP-seq_ENCFF814BNR_signal,stomach_16.5-days_embryonic_H3K4me2_ChIP-seq_ENCFF501CJA_width,stomach_16.5-days_embryonic_H3K4me2_ChIP-seq_ENCFF501CJA_signal,stomach_16.5-days_embryonic_H3K4me3_ChIP-seq_ENCFF569KWB_width,stomach_16.5-days_embryonic_H3K4me3_ChIP-seq_ENCFF569KWB_signal,stomach_16.5-days_embryonic_H3K9ac_ChIP-seq_ENCFF068FWP_width,stomach_16.5-days_embryonic_H3K9ac_ChIP-seq_ENCFF068FWP_signal,stomach_16.5-days_embryonic_H3K9me3_ChIP-seq_ENCFF544RGQ_width,stomach_16.5-days_embryonic_H3K9me3_ChIP-seq_ENCFF544RGQ_signal
0,Xkr4,0.260962,0.369031,-1.299443,0.436612,-0.141153,-0.246419,-0.095730,Partly affected unmarked,2.82776,...,459.0,4.17547,2522.0,32.56543,2456.0,37.44113,1852.0,6.81303,NaN,NaN
1,Sox17,0.407478,-0.646836,0.337369,0.092069,0.278734,0.488633,0.088057,Partly affected marked,5.30733,...,702.0,4.40404,1327.0,17.05396,1257.0,14.96443,507.0,4.75484,NaN,NaN
2,Mrpl15,1.325021,1.099265,-0.278983,-0.551568,-0.132808,-0.126111,0.041484,Partly affected unmarked,0.00000,...,479.0,2.85033,1334.0,32.48818,2048.0,41.08784,1396.0,18.10043,NaN,NaN
3,Tcea1,1.577630,1.028430,-0.621913,-0.289489,-0.151392,-0.142735,0.015326,Partly affected unmarked,0.00000,...,NaN,NaN,1381.0,12.89607,2526.0,55.09949,1176.0,12.09817,NaN,NaN
4,Rgs20,0.316957,1.250120,-0.743550,-0.290937,-0.094171,-0.466210,0.249359,Partly affected unmarked,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12792,Frmpd4,-0.385455,0.081109,-1.109779,0.392005,-0.463031,-0.259205,-0.120672,Partly affected unmarked,3.20311,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12793,Msl3,1.032435,0.199148,-0.023354,-0.406618,-0.192482,-0.072598,-0.112576,Partly affected unmarked,2.93122,...,NaN,NaN,2402.0,21.82412,3127.0,33.87626,2257.0,13.40337,NaN,NaN
12794,Arhgap6,-0.129757,-0.457979,0.349736,0.978575,0.381812,0.111678,-0.803318,Partly affected marked,3.56165,...,NaN,NaN,2878.0,23.78164,2420.0,18.13809,2004.0,5.96359,NaN,NaN
12795,Hccs,1.083603,0.476223,-0.351159,-0.236171,-0.180150,-0.213555,-0.096767,Partly affected unmarked,3.30158,...,NaN,NaN,1698.0,17.11209,2270.0,30.84611,1158.0,12.58762,NaN,NaN


In [58]:
df_pos_subset = df_all[~df_all['GroupLabel'].isin(['Unmarked immune response', 'Partly affected unmarked', 'Partly affected marked', 'Consistently affected marked', 'Consistently affected unmarked'])]
df_pos_subset['GroupLabel'].value_counts()

Unmarked proliferation                                             111
Marked Anterior                                                     94
Development                                                         93
Marked Posterior                                                    44
Unmarked proliferation Unmarked immune response                     34
Marked Posterior Development                                        30
Unmarked proliferation Development                                  21
Unmarked immune response Development                                12
Unmarked proliferation Marked Anterior                              12
Unmarked immune response Marked Anterior                             7
Marked Posterior Marked Anterior                                     6
Unmarked proliferation Unmarked immune response Marked Anterior      2
Name: GroupLabel, dtype: int64

In [59]:
df_neg_subset = df_all[df_all['GroupLabel'].isin(['Consistently affected marked', 'Consistently affected unmarked'])]
print(df_neg_subset['GroupLabel'].value_counts())

Consistently affected marked      437
Consistently affected unmarked    301
Name: GroupLabel, dtype: int64


In [83]:
mouse_tfs = pd.read_csv(f'../data/doro_mouse.csv')
mouse_tfs = mouse_tfs[mouse_tfs['confidence'].isin(['A'])]
tfs = set(mouse_tfs['tf'].values)

In [84]:
df_training = df_all[['external_gene_name'] + [c for c in df_all.columns if 'H3' in c and '_' in c and 'signal' in c]]
df_training = df_training.fillna(0)

In [85]:
df_pos = df_training[df_training['external_gene_name'].isin(df_pos_subset['external_gene_name'].values)]
df_pos.set_index('external_gene_name', inplace=True)
df_pos = df_pos.T.sample(frac=1) # Randomly shuffle the data

df_neg = df_training[df_training['external_gene_name'].isin(df_neg_subset['external_gene_name'].values)]
df_neg.set_index('external_gene_name', inplace=True)
df_neg = df_neg.T.sample(frac=1)

df_tfs = df_training[df_training['external_gene_name'].isin(tfs)]
df_tfs.set_index('external_gene_name', inplace=True)
df_tfs = df_tfs.T.sample(frac=1)

In [86]:
df_pos.shape

(588, 466)

In [87]:
from scivae import Validate
from sciutil import SciUtil

label = 'time'
label_i = 1
df_pos_labels = [i.split('_')[label_i] for i in df_pos.index]
df_neg_labels = [i.split('_')[label_i] for i in df_neg.index]
df_tfs_labels = [i.split('_')[label_i] for i in df_tfs.index]
print(set(df_tfs_labels))


u = SciUtil()
u.dp([f'VAE identified genes on {label}:', df_pos.shape])
vd = Validate(df_pos.values, df_pos_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))

u.dp([f'TFs identified genes on {label}:', df_tfs.shape])
vd = Validate(df_tfs.values, df_tfs_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))

u.dp([f'Consistently affected genes on {label}:', df_neg.shape])
vd = Validate(df_neg.values, df_neg_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))


{'8-weeks', '11.5-days', '16.5-days', '15.5-days', '0-days', '10.5-days', '12.5-days', '13.5-days', '14.5-days'}
--------------------------------------------------------------------------------
                   VAE identified genes on time:	(588, 466)	                    
--------------------------------------------------------------------------------
0.4191086691086692
0.1009920634920635
--------------------------------------------------------------------------------
                    TFs identified genes on time:	(588, 84)	                    
--------------------------------------------------------------------------------
0.2709435626102292
0.13086419753086417
--------------------------------------------------------------------------------
                Consistently affected genes on time:	(588, 738)	                
--------------------------------------------------------------------------------
0.29360269360269364
0.12222222222222223


In [91]:
from scivae import Validate
from sciutil import SciUtil

label = 'time'
label_i = 2
df_pos_labels = [i.split('_')[label_i] for i in df_pos.index]
df_neg_labels = [i.split('_')[label_i] for i in df_neg.index]
df_tfs_labels = [i.split('_')[label_i] for i in df_tfs.index]
print(set(df_tfs_labels))


u = SciUtil()
u.dp([f'VAE identified genes on {label}:', df_pos.shape])
vd = Validate(df_pos.values, df_pos_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))

u.dp([f'TFs identified genes on {label}:', df_tfs.shape])
vd = Validate(df_tfs.values, df_tfs_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))

u.dp([f'Consistently affected genes on {label}:', df_neg.shape])
vd = Validate(df_neg.values, df_neg_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))


{'embryonic', 'adult', 'postnatal'}
--------------------------------------------------------------------------------
                   VAE identified genes on time:	(588, 466)	                    
--------------------------------------------------------------------------------
0.32882882882882886
0.3333333333333333
--------------------------------------------------------------------------------
                    TFs identified genes on time:	(588, 84)	                    
--------------------------------------------------------------------------------
0.5
0.3333333333333333
--------------------------------------------------------------------------------
                Consistently affected genes on time:	(588, 738)	                
--------------------------------------------------------------------------------
0.35555555555555557
0.3333333333333333


In [88]:

label = 'tissue'
label_i = 0
df_pos_labels = [i.split('_')[label_i] for i in df_pos.index]
df_neg_labels = [i.split('_')[label_i] for i in df_neg.index]
df_tfs_labels = [i.split('_')[label_i] for i in df_tfs.index]
print(set(df_tfs_labels))


u = SciUtil()
u.dp([f'VAE identified genes on {label}:'])
vd = Validate(df_pos.values, df_pos_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))

u.dp([f'TFs identified genes on {label}:'])
vd = Validate(df_tfs.values, df_tfs_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))

u.dp([f'Consistently affected genes on {label}:'])
vd = Validate(df_neg.values, df_neg_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))


{'embryonic-facial-prominence', 'neural-tube', 'hindbrain', 'intestine', 'lung', 'kidney', 'forebrain', 'liver', 'midbrain', 'limb', 'stomach', 'heart'}
--------------------------------------------------------------------------------
                        VAE identified genes on tissue:	                        
--------------------------------------------------------------------------------
0.6137784601019894
0.4600184945773181
--------------------------------------------------------------------------------
                        TFs identified genes on tissue:	                        
--------------------------------------------------------------------------------
0.24431216931216929
0.1597222222222222
--------------------------------------------------------------------------------
                    Consistently affected genes on tissue:	                     
--------------------------------------------------------------------------------
0.31620370370370365
0.1529040404040404


In [89]:

label = 'mark'
label_i = 3
df_pos_labels = [i.split('_')[label_i] for i in df_pos.index]
df_neg_labels = [i.split('_')[label_i] for i in df_neg.index]
df_tfs_labels = [i.split('_')[label_i] for i in df_tfs.index]
print(set(df_tfs_labels))


u = SciUtil()
u.dp([f'VAE identified genes on {label}:'])
vd = Validate(df_pos.values, df_pos_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))

u.dp([f'TFs identified genes on {label}:'])
vd = Validate(df_tfs.values, df_tfs_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))

u.dp([f'Consistently affected genes on {label}:'])
vd = Validate(df_neg.values, df_neg_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))


{'H3K9ac', 'H3K27me3', 'H3K4me1', 'H3K4me2', 'H3K9me3', 'H3K36me3', 'H3K27ac', 'H3K4me3'}
--------------------------------------------------------------------------------
                         VAE identified genes on mark:	                         
--------------------------------------------------------------------------------
0.971875
0.9782986111111112
--------------------------------------------------------------------------------
                         TFs identified genes on mark:	                         
--------------------------------------------------------------------------------
0.979020979020979
0.9903846153846154
--------------------------------------------------------------------------------
                     Consistently affected genes on mark:	                      
--------------------------------------------------------------------------------
0.9696691176470589
0.9527007918552036


## Look at human brain expression

In [ ]:
# Get a map from mouse to human
map_df = pd.read_csv('../data/mm10_orthologs_hsapiens_gene_ensembl-GRCh38.p13.csv')
map_df = map_df.dropna()
plt.hist(map_df['mmusculus_homolog_perc_id_r1'].values)
plt.show()

# Keep ones with over 80%
mapping = map_df[map_df['mmusculus_homolog_perc_id_r1'] > 80]
mapping = dict(zip(mapping.mmusculus_homolog_ensembl_gene, mapping.ensembl_gene_id))

# Now map each of the sets to the human version
gene_name_to_ensembl = pd.read_csv('../data/mmusculus_gene_ensembl-GRCm39.csv')
gene_name_to_ensembl = dict(zip(gene_name_to_ensembl['external_gene_name'].values, gene_name_to_ensembl['ensembl_gene_id'].values))

# For each of them make the map
df_neg_human = [gene_name_to_ensembl.get(g) for g in df_neg_subset.external_gene_name.values]
df_neg_human = [mapping.get(g) for g in df_neg_human if g is not None]
df_neg_human = [g for g in df_neg_human if g is not None]

df_pos_human = [gene_name_to_ensembl.get(g) for g in df_pos_subset.external_gene_name.values]
df_pos_human = [mapping.get(g) for g in df_pos_human if g is not None]
df_pos_human = [g for g in df_pos_human if g is not None]

print(len(df_neg_human), len(df_pos_human))

# For TFs we just use human TFs from dorothea
human_tfs = pd.read_csv('../data/dorothea_hs_ABCD.csv')
human_tfs = human_tfs[human_tfs['confidence'].isin(['A'])]
human_tfs = set(human_tfs['tf'].values)
print(len(human_tfs))

In [138]:
from statannot import add_stat_annotation
data_dir = '../data/'
r_dir = f'{data_dir}results/deseq2/'
supp_dir = f'{data_dir}input/supps/'

human_b = pd.read_csv(supp_dir + 'genes_matrix_csv/expression_matrix.csv', header=None)
human_rm = pd.read_csv(supp_dir + 'genes_matrix_csv/rows_metadata.csv')
human_cm = pd.read_csv(supp_dir + 'genes_matrix_csv/columns_metadata.csv')

ages = human_cm['age'].values
gender = human_cm['gender'].values
regions = human_cm['structure_acronym'].values
col_names = ['row-id']
donor_id = human_cm['donor_id'].values

# Rename columns to early, mid, w1 = e8, 9, w2 = 12, 13, w3 = 16, 17, w4 = > 17
for i, r in enumerate(regions):
    a = int(ages[i].split(' ')[0])
    stage = 0
    if a > 17:
        stage = 'W4'
    elif a > 13:
        stage = 'W3'
    elif a > 9:
        stage = 'W2'
    else:
        stage = 'W1'
    
    cn = regions[i] + '_' + ages[i].replace(' ', '-') + '_' + stage + '_' + gender[i]  + '_' + str(donor_id[i])
    col_names.append(cn)

human_b.columns = col_names
human_emb = human_b[[c for c in human_b.columns if 'pcw' in c]]# and ('OFC' in c or 'MD' in c or 'CBC' in c)]]

human_emb['ensembl_gene_id'] = human_rm['ensembl_gene_id'].values
human_emb['gene_symbol'] = human_rm['gene_symbol'].values
human_emb['entrez_id'] = human_rm['entrez_id'].values

human_emb_a = pd.DataFrame()
human_emb_num = pd.DataFrame()
human_emb_a['ensembl_gene_id'] = human_rm['ensembl_gene_id'].values
human_emb = human_emb.fillna(0)
for c in human_emb.columns:
    if 'entrez_id' not in c:
        try:
            human_emb_a[c] = np.log2(human_emb[c].values + 1)
            human_emb_num[c] = np.log2(human_emb[c].values + 1)
        except:
            print(c)

/var/folders/gq/6ljhmvm1713fykdjqbl188pm0000gn/T/ipykernel_13767/1843951305.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  human_emb['ensembl_gene_id'] = human_rm['ensembl_gene_id'].values
/var/folders/gq/6ljhmvm1713fykdjqbl188pm0000gn/T/ipykernel_13767/1843951305.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  human_emb['gene_symbol'] = human_rm['gene_symbol'].values
/var/folders/gq/6ljhmvm1713fykdjqbl188pm0000gn/T/ipykernel_13767/1843951305.py:37: SettingWithCopyWarning: 
A value is trying to b

Ocx_8-pcw_W1_M_13058
M1C-S1C_8-pcw_W1_M_13058
AMY_8-pcw_W1_M_13058
MGE_8-pcw_W1_M_13058
STC_8-pcw_W1_M_13058
URL_8-pcw_W1_M_13058
CGE_8-pcw_W1_M_13058
DTH_8-pcw_W1_M_13058
MFC_8-pcw_W1_M_13058
DFC_8-pcw_W1_M_13058
OFC_8-pcw_W1_M_13058
LGE_8-pcw_W1_M_13058
ITC_8-pcw_W1_M_13058
HIP_8-pcw_W1_M_13058
VFC_8-pcw_W1_M_13058
PCx_8-pcw_W1_M_13058
DFC_9-pcw_W1_M_12833
MFC_9-pcw_W1_M_12833
AMY_9-pcw_W1_M_12833
DTH_9-pcw_W1_M_12833
URL_9-pcw_W1_M_12833
LGE_9-pcw_W1_M_12833
M1C-S1C_9-pcw_W1_M_12833
MGE_9-pcw_W1_M_12833
TCx_9-pcw_W1_M_12833
HIP_9-pcw_W1_M_12833
Ocx_9-pcw_W1_M_12833
CGE_9-pcw_W1_M_12833
OFC_9-pcw_W1_M_12833
PCx_9-pcw_W1_M_12833
HIP_12-pcw_W2_F_12835
DTH_12-pcw_W2_F_12835
AMY_12-pcw_W2_F_12835
A1C_12-pcw_W2_F_12835
V1C_12-pcw_W2_F_12835
MFC_12-pcw_W2_F_12835
STR_12-pcw_W2_F_12835
M1C_12-pcw_W2_F_12835
DFC_12-pcw_W2_F_12835
ITC_12-pcw_W2_F_12835
VFC_12-pcw_W2_F_12835
STC_12-pcw_W2_F_12835
IPC_12-pcw_W2_F_12835
OFC_12-pcw_W2_F_12835
S1C_12-pcw_W2_F_12835
STC_12-pcw_W2_F_12960
V1C_12-pcw

In [139]:
df_pos_h = human_emb[human_emb['ensembl_gene_id'].isin(df_pos_human)]
df_pos_h.set_index('gene_symbol', inplace=True)
df_pos_h = df_pos_h[[c for c in df_pos_h.columns if 'entrez' not in c and 'ensembl' not in c]]
df_pos_h = df_pos_h.T.sample(frac=1) # Randomly shuffle the data

df_neg_h = human_emb[human_emb['ensembl_gene_id'].isin(df_neg_human)]
df_neg_h.set_index('gene_symbol', inplace=True)
df_neg_h = df_neg_h[[c for c in df_neg_h.columns if 'entrez' not in c and 'ensembl' not in c]]

df_neg_h = df_neg_h.T.sample(frac=1)

df_tfs_h = human_emb[human_emb['gene_symbol'].isin(human_tfs)]
df_tfs_h.set_index('gene_symbol', inplace=True)
df_tfs_h = df_tfs_h[[c for c in df_tfs_h.columns if 'entrez' not in c and 'ensembl' not in c]]

df_tfs_h = df_tfs_h.T.sample(frac=1)

In [140]:
from scivae import Validate
from sciutil import SciUtil

label = 'tissue'
label_i = 0
df_pos_labels = [i.split('_')[label_i] for i in df_pos_h.index]
df_neg_labels = [i.split('_')[label_i] for i in df_neg_h.index]
df_tfs_labels = [i.split('_')[label_i] for i in df_tfs_h.index]
print(set(df_tfs_labels))


u = SciUtil()
u.dp([f'VAE identified genes on {label}:', df_pos.shape])
vd = Validate(df_pos_h.values, df_pos_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))

u.dp([f'TFs identified genes on {label}:', df_tfs.shape])
vd = Validate(df_tfs_h.values, df_tfs_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))

u.dp([f'Consistently affected genes on {label}:', df_neg.shape])
vd = Validate(df_neg_h.values, df_neg_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))


{'AMY', 'M1C', 'DTH', 'HIP', 'URL', 'S1C', 'Ocx', 'CB', 'DFC', 'M1C-S1C', 'VFC', 'STR', 'IPC', 'CBC', 'ITC', 'TCx', 'V1C', 'LGE', 'CGE', 'PCx', 'MFC', 'A1C', 'OFC', 'MGE', 'STC', 'MD'}
--------------------------------------------------------------------------------
                  VAE identified genes on tissue:	(588, 466)	                   
--------------------------------------------------------------------------------
0.4
0.1
--------------------------------------------------------------------------------
                   TFs identified genes on tissue:	(588, 84)	                   
--------------------------------------------------------------------------------
0.1398148148148148
0.12037037037037036
--------------------------------------------------------------------------------
               Consistently affected genes on tissue:	(588, 738)	               
--------------------------------------------------------------------------------
0.2490196078431372
0.17647058823529413


/Users/ariane/opt/miniconda3/envs/scivae_10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/ariane/opt/miniconda3/envs/scivae_10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/ariane/opt/miniconda3/envs/scivae_10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [141]:
from scivae import Validate
from sciutil import SciUtil

label = 'age'
label_i = 1
df_pos_labels = [i.split('_')[label_i] for i in df_pos_h.index]
df_neg_labels = [i.split('_')[label_i] for i in df_neg_h.index]
df_tfs_labels = [i.split('_')[label_i] for i in df_tfs_h.index]
print(set(df_tfs_labels))


u = SciUtil()
u.dp([f'VAE identified genes on {label}:', df_pos.shape])
vd = Validate(df_pos_h.values, df_pos_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))

u.dp([f'TFs identified genes on {label}:', df_tfs.shape])
vd = Validate(df_tfs_h.values, df_tfs_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))

u.dp([f'Consistently affected genes on {label}:', df_neg.shape])
vd = Validate(df_neg_h.values, df_neg_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))


{'12-pcw', '24-pcw', '25-pcw', '26-pcw', '16-pcw', '19-pcw', '21-pcw', '8-pcw', '17-pcw', '9-pcw', '35-pcw', '13-pcw', '37-pcw'}
--------------------------------------------------------------------------------
                    VAE identified genes on age:	(588, 466)	                    
--------------------------------------------------------------------------------
0.8001443001443
0.22857142857142856
--------------------------------------------------------------------------------
                    TFs identified genes on age:	(588, 84)	                     
--------------------------------------------------------------------------------
0.59004329004329
0.29696969696969694
--------------------------------------------------------------------------------
                Consistently affected genes on age:	(588, 738)	                 
--------------------------------------------------------------------------------
0.6833333333333333
0.41666666666666663


In [143]:
from scivae import Validate
from sciutil import SciUtil

label = 'age'
label_i = 2
df_pos_labels = [i.split('_')[label_i] for i in df_pos_h.index]
df_neg_labels = [i.split('_')[label_i] for i in df_neg_h.index]
df_tfs_labels = [i.split('_')[label_i] for i in df_tfs_h.index]
print(set(df_tfs_labels))


u = SciUtil()
u.dp([f'VAE identified genes on {label}:', df_pos.shape])
vd = Validate(df_pos_h.values, df_pos_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))

u.dp([f'TFs identified genes on {label}:', df_tfs.shape])
vd = Validate(df_tfs_h.values, df_tfs_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))

u.dp([f'Consistently affected genes on {label}:', df_neg.shape])
vd = Validate(df_neg_h.values, df_neg_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))


{'W4', 'W1', 'W2', 'W3'}
--------------------------------------------------------------------------------
                    VAE identified genes on age:	(588, 466)	                    
--------------------------------------------------------------------------------
0.791517857142857
0.5563988095238095
--------------------------------------------------------------------------------
                    TFs identified genes on age:	(588, 84)	                     
--------------------------------------------------------------------------------
0.7840909090909091
0.4409090909090909
--------------------------------------------------------------------------------
                Consistently affected genes on age:	(588, 738)	                 
--------------------------------------------------------------------------------
0.94375
0.81875


In [142]:
from scivae import Validate
from sciutil import SciUtil

label = 'sex'
label_i = 3
df_pos_labels = [i.split('_')[label_i] for i in df_pos_h.index]
df_neg_labels = [i.split('_')[label_i] for i in df_neg_h.index]
df_tfs_labels = [i.split('_')[label_i] for i in df_tfs_h.index]
print(set(df_tfs_labels))


u = SciUtil()
u.dp([f'VAE identified genes on {label}:', df_pos.shape])
vd = Validate(df_pos_h.values, df_pos_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))

u.dp([f'TFs identified genes on {label}:', df_tfs.shape])
vd = Validate(df_tfs_h.values, df_tfs_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))

u.dp([f'Consistently affected genes on {label}:', df_neg.shape])
vd = Validate(df_neg_h.values, df_neg_labels)
print(vd.predict('rf', 'balanced_accuracy'))
print(vd.predict('svm', 'balanced_accuracy'))


{'M', 'F'}
--------------------------------------------------------------------------------
                    VAE identified genes on sex:	(588, 466)	                    
--------------------------------------------------------------------------------
0.825
0.5
--------------------------------------------------------------------------------
                    TFs identified genes on sex:	(588, 84)	                     
--------------------------------------------------------------------------------
0.8307692307692307
0.676923076923077
--------------------------------------------------------------------------------
                Consistently affected genes on sex:	(588, 738)	                 
--------------------------------------------------------------------------------
0.8701298701298701
0.8928571428571428
